**운전자 이상행동 데이터셋 다운로드 및 변환**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
%cd /content/drive/MyDrive/CAB/CAB_dataset/JPEGImages
!unzip -qq "/content/drive/MyDrive/CAB/CAB_dataset/JPEGImages/images_bbox.zip"

/content/drive/MyDrive/CAB/CAB_dataset/JPEGImages


In [66]:
%cd /content/drive/MyDrive/CAB/CAB_dataset/Annotations
!unzip -qq "/content/drive/MyDrive/CAB/CAB_dataset/Annotations/label_bbox.zip"

/content/drive/MyDrive/CAB/CAB_dataset/temp
unzip:  cannot find or open /content/drive/MyDrive/CAB/CAB_dataset/temp/졸음운전 예방을 위한 운전자 상태 정보 영상, /content/drive/MyDrive/CAB/CAB_dataset/temp/졸음운전 예방을 위한 운전자 상태 정보 영상.zip or /content/drive/MyDrive/CAB/CAB_dataset/temp/졸음운전 예방을 위한 운전자 상태 정보 영상.ZIP.


파일 경로 변환
- /content/drive/MyDrive/CAB/CAB_dataset/JPEGImages 하위 디렉토리에 있는 이미지를 /content/drive/MyDrive/CAB/CAB_dataset/JPEGImages로 옮김

In [ ]:
import os
import shutil

def move_file(base_dir):
  for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.lower().endswith(('.jpg', 'json')):
            source_path = os.path.join(root, file)
            destination_path = os.path.join(base_dir, file)

            shutil.move(source_path, destination_path)

jpeg_dir = "/content/drive/MyDrive/CAB/CAB_dataset/JPEGImages"
annot_dir = "/content/drive/MyDrive/CAB/CAB_dataset/Annotations"

move_file(jpeg_dir)
move_file(annot_dir)

json 파일 형태를 YOLOv5 훈련에 사용할 수 있도록 txt로 변환한다.

In [ ]:
class_mapping = {
      "Face": 0,
      "Leye": 1,
      "Reye": 2,
      "Mouth": 3,
      "Cigar": 4,
      "Phone": 5
}

In [78]:
import json
import os

source_path = "/content/drive/MyDrive/CAB/CAB_dataset/Annotations/"
dest_path = "/content/drive/MyDrive/CAB/CAB_dataset/TextAnnotations/"

# 지정된 경로에 있는 모든 파일과 디렉토리를 가져옴
json_files = [f for f in os.listdir(source_path) if f.endswith('.json')]

for json_file in json_files:
  file_path = os.path.join(source_path, json_file)

  with open(file_path, 'r', encoding='utf-8') as f:
      data = json.load(f)

  file_name = data["FileInfo"]["FileName"].split(".")[0]

  # 이미지 크기
  img_width = data["FileInfo"]["Width"]
  img_height = data["FileInfo"]["Height"]

  # txt 파일을 생성할 리스트
  yolo_data = []

  # 바운딩 박스를 확인하여 YOLO 형식으로 변환
  for obj, bbox in data["ObjectInfo"]["BoundingBox"].items():
      if bbox["isVisible"]:
          class_id = class_mapping[obj]
          xmin, ymin, xmax, ymax = bbox["Position"]

          # YOLO 형식의 바운딩 박스 (중앙 좌표와 크기) 계산
          x_center = (xmin + xmax) / 2 / img_width
          y_center = (ymin + ymax) / 2 / img_height
          width = (xmax - xmin) / img_width
          height = (ymax - ymin) / img_height

          # YOLO 포맷: 클래스 ID와 정규화된 좌표
          yolo_data.append(f"{class_id} {x_center} {y_center} {width} {height}")

  output_file = os.path.join(dest_path, file_name + ".txt")

  with open(output_file, "w") as f:
      for line in yolo_data:
          f.write(line + "\n")

다음으로, 모든 Image의 절대 경로가 적힌 리스트를 만든다.

In [94]:
base_path = "/content/drive/MyDrive/CAB/CAB_dataset/JPEGImages"
img_files = []

for file in os.listdir(base_path):
  if file.endswith(".jpg"):
    file_path = os.path.join(base_path, file)
    img_files.append(file_path)

yolo 학습을 위해 annotation 파일과 image 파일을 한 폴더로 옮긴다.

In [87]:
img_source_path = "/content/drive/MyDrive/CAB/CAB_dataset/JPEGImages/"
txt_source_path = "/content/drive/MyDrive/CAB/CAB_dataset/TextAnnotations/"
dest_path = "/content/drive/MyDrive/CAB/CAB_dataset/TxtImg"

for f in os.listdir(img_source_path):
  if f.endswith(".jpg"):
    shutil.move(os.path.join(img_source_path, f), dest_path)

for f in os.listdir(txt_source_path):
  if f.endswith(".txt"):
    shutil.move(os.path.join(txt_source_path, f), dest_path)

전체 이미지를 6:2:2 비율로 나누어 train set, validation set, test set을 생성한다.


In [90]:
data_root = "/content/drive/MyDrive/CAB/CAB_dataset/"

train_root = os.path.join(data_root, "train")
val_root = os.path.join(data_root, "val")
test_root = os.path.join(data_root, "test")

os.makedirs(train_root, exist_ok=True)
os.makedirs(val_root, exist_ok=True)
os.makedirs(test_root, exist_ok=True)

In [96]:
import random
from tqdm import tqdm

random.shuffle(img_files)

for idx, img_src in tqdm(enumerate(img_files)):
  txt_src = os.path.splitext(img_src)[0] + ".txt"
  img_name = os.path.split(img_src)[-1]
  text_name = os.path.split(txt_src)[-1]

  if idx % 10 < 6:
    img_dst = os.path.join(train_root, img_name)
    txt_dst = os.path.join(train_root, text_name)
  elif idx % 10 < 8:
    img_dst = os.path.join(val_root, img_name)
    txt_dst = os.path.join(val_root, text_name)
  else:
    img_dst = os.path.join(test_root, img_name)
    txt_dst = os.path.join(test_root, text_name)

  shutil.copy2(img_src, img_dst)
  shutil.copy2(txt_src, txt_dst)

12563it [09:05, 23.02it/s]
